In [ ]:
# default_exp postprocessing

Module Postprocessing
---

Nipype interfaces for generating connectomes after pre-processing of DWI and `.trk` generation.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#exporti

import os

from nipype import IdentityInterface
from nipype.pipeline import Node, Workflow
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, File, TraitedSpec, traits
from nipype.interfaces.mrtrix3.connectivity import LabelConvert
from nipype.interfaces.mrtrix3.utils import Generate5tt
from nipype.interfaces.mrtrix3.preprocess import ResponseSD
from nipype.interfaces.mrtrix3.reconst import EstimateFOD

### Atlas files input:

Use `IdentityInterface` as a node for atlas inputs. Atlas inputs as a list if constructing multiple connnectomes. Look-up tables should also be an input. 

https://miykael.github.io/nipype_tutorial/notebooks/basic_data_input.html

In [ ]:
#example
#usage

atlas_dir = '/Users/xxie/lab/Human_Brain_Atlases'  # needs to be defined... either as an user input or pre-defined BIDS dir
atlas_list = ['brainnectome', 'desikan-killiany']  # probably an input as well, currently have LUT corrected images for desikan-killiany, brainnectome, aal

# Node for IdentityInterface
atlas_source = Node(IdentityInterface(fields = ['atlas_name']), 
                    name = 'atlas_source')
atlas_source.iterables = [('atlas_name', atlas_list)]  # iterate over atlas input names

# Node for select files: actually selecting files
atlas_template = {
    'atlases': os.path.join(
        atlas_dir, '{atlas_name}', '*_1mm.nii.gz' 
    )
}
atlas_file = Node(
    SelectFiles(atlas_template),
    base_directory=atlas_dir,
    name='select_atlas'
)

# Create datasink for selected files
datasink = Node(DataSink(base_directory = '/Users/xxie/lab/pipetography/outputs',
               container='datasink'),
                name='datasink')

wf_atlas = Workflow(name="choose_atlas")
wf_atlas.connect(atlas_source, 'atlas_name', atlas_file, 'atlas_name')
wf_atlas.connect(atlas_file, 'atlases', datasink, 'atlases')
wf_atlas.run()

! ls -lh /Users/xxie/lab/pipetography/outputs/datasink/atlases/

201001-10:14:41,949 nipype.workflow INFO:
	 Workflow choose_atlas settings: ['check', 'execution', 'logging', 'monitoring']
201001-10:14:41,960 nipype.workflow INFO:
	 Running serially.
201001-10:14:41,961 nipype.workflow INFO:
	 [Node] Setting-up "choose_atlas.select_atlas" in "/private/var/folders/26/ys2mhp6j58n047rd6tvx76sm0000gp/T/tmpwdjftgxd/choose_atlas/_atlas_name_desikan-killiany/select_atlas".
201001-10:14:41,966 nipype.workflow INFO:
	 [Node] Running "select_atlas" ("nipype.interfaces.io.SelectFiles")
201001-10:14:41,972 nipype.workflow INFO:
	 [Node] Finished "choose_atlas.select_atlas".
201001-10:14:41,973 nipype.workflow INFO:
	 [Node] Setting-up "choose_atlas.datasink" in "/private/var/folders/26/ys2mhp6j58n047rd6tvx76sm0000gp/T/tmp_gqjektk/choose_atlas/_atlas_name_desikan-killiany/datasink".
201001-10:14:41,980 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
201001-10:14:41,989 nipype.workflow INFO:
	 [Node] Finished "choose_atlas.data

### Subject files input:

Tracts `.tck` files input (need to convert from trk to tck if using NVIDIA cuda, currently included in shell script attached to NVIDIA singularity).

Whole brain volume `T1.mgz` files input, these are outputs of freesurfer's `recon-all`, subsequently used to generate tissue segmented files. 

In [ ]:
data_dir = '/Users/xxie/sample_data/dwipreproc'
session_id = '002'
subject_id = '01'

subj_source = Node(IdentityInterface(fields=["subject_id", "session_id"]), name = 'subj_source')
subj_template = {
    'tck': os.path.join(data_dir, 'cuda_tracking', '_session_id_{session_id}_subject_id_{subject_id}', 'sub-{subject_id}_ses-{session_id}.tck'),
    'dwi': os.path.join(data_dir, 'derivatives', 'dwi_acpc_aligned_1mm', '_session_id_{session_id}_subject_id_{subject_id}', 'dwi_acpc_1mm.mif'),
    'acpcT1':  os.path.join(data_dir, 'derivatives', 't1_acpc_aligned', '_session_id_{session_id}_subject_id_{subject_id}', 'acpc_t1.nii')
}
subj_source.inputs.subject_id = subject_id
subj_source.inputs.session_id = session_id



201001-11:20:16,355 nipype.workflow INFO:
	 [Node] Setting-up "subj_source" in "/private/var/folders/26/ys2mhp6j58n047rd6tvx76sm0000gp/T/tmpd8y9s0wa/subj_source".
201001-11:20:16,360 nipype.workflow INFO:
	 [Node] Running "subj_source" ("nipype.interfaces.utility.base.IdentityInterface")
201001-11:20:16,365 nipype.workflow INFO:
	 [Node] Finished "subj_source".



session_id = 002
subject_id = 01

### ANTS Registration

Align atlases to DWI images

In [ ]:
#example
#usage

linear_reg = Node(ants.Registration(), name = 'linear_Registration')
# Registration inputs:
#linear_reg.inputs.moving_image = atlas_dir #need DIR to atlas
linear_reg.inputs.output_transform_prefix = 'atlas_in_dwi_affine'
linear_reg.inputs.dimension = 3 #-d
# -tranform
linear_reg.inputs.collapse_output_transforms = True # -z flag
linear_reg.inputs.transforms = ['Affine']
linear_reg.inputs.transform_parameters=[(0.1,)]
# -metric
linear_reg.inputs.metric = ['MI']
linear_reg.inputs.metric_weight = [1] #default, value ignored by ANTS
linear_reg.inputs.radius_or_number_of_bins = [64]
# -convergence
linear_reg.inputs.number_of_iterations = [[500,200,200,100]]
linear_reg.inputs.convergence_threshold = [1e-06]
linear_reg.inputs.convergence_window_size = [10]
# -s
linear_reg.inputs.smoothing_sigmas = [[4,2,1,0]]
linear_reg.inputs.sigma_units = ['vox']
# -f
linear_reg.inputs.shrink_factors = [[8,4,2,1]]
linear_reg.inputs.use_histogram_matching = [True] # -u flag
linear_reg.inputs.output_warped_image = 'atlas_in_dwi_affine.nii.gz'

In [ ]:
#example
#usage

#### additional nonlinear registration node:
syn_reg = Node(ants.Registration(), name = 'nonlinear_registration')
# Registration inputs:
#syn_reg.inputs.moving_image = 'interface_testing/atlas_in_dwi_affine.nii.gz' #need DIR to atlas
syn_reg.inputs.output_transform_prefix = 'atlas_in_dwi_syn'
syn_reg.inputs.dimension = 3 #-d
# -tranform
syn_reg.inputs.collapse_output_transforms = True # -z flag
syn_reg.inputs.transforms = ['SyN']
syn_reg.inputs.transform_parameters=[(0.1,)]
# -metric
syn_reg.inputs.metric = ['MI']
syn_reg.inputs.metric_weight = [1] #default, value ignored by ANTS
syn_reg.inputs.radius_or_number_of_bins = [64]
# -convergence
syn_reg.inputs.number_of_iterations = [[500,200,200,100]]
syn_reg.inputs.convergence_threshold = [1e-06]
syn_reg.inputs.convergence_window_size = [10]
# -s
syn_reg.inputs.smoothing_sigmas = [[4,2,1,0]]
syn_reg.inputs.sigma_units = ['vox']
# -f
syn_reg.inputs.shrink_factors = [[8,4,2,1]]
syn_reg.inputs.use_histogram_matching = [True] # -u flag
syn_reg.inputs.output_warped_image = 'atlas_in_dwi_syn.nii.gz'

### FOD Modeling

Create white matter fiber orientation image. First use `dwi2response dhollander` to generate white matter, grey matter, and CSF spherical deconvolution response files, then use `dwi2fod msmt_csd` with a tissue response function input to generate FODs for the white matter, gray matter, and CSF.

In [ ]:
#dwi2response
#dwi2fod

### Mrtrix3 - SIFT2

Spherical-deconvolution Informed Filtering of Tractograms (SIFT) `tcksift2`:

In [ ]:
#exporti
class tckSIFT2InputSpec(CommandLineInputSpec):
    in_file = File(
        exists=True,
        mandatory=True,
        argstr="%s",
        position=-2,
        desc="input track file"
    )
    in_fod = File(
        exists=True,
        mandatory=True,
        argstr="%s",
        position=-1,
        desc="input image containing the spherical harmonics of the fiber orientation distributions"
    )
    proc_mask = File(
        exists=True,
        argstr="-proc_mask %s",
        desc="provide an image containing the processing mask weights for the model; image spatial dimensions must match the fixel image"
    )
    act = File(
        exists=True,
        argstr="-act %s",
        desc="use an ACT five-tissue-type segmented anatomical image to derive the processing mask"
    )
    fd_scale_gm = traits.Bool(
        argstr="-fd_scale_gm",
        desc="in conjunction with -act to heuristically downsize the fibre density estimates based on the presence of GM in the voxel. This can assist in reducing tissue interface effects when using a single-tissue deconvolution algorithm"
    )
    nthreads = traits.Int(
        argstr="-nthreads %d",
        desc="number of threads. if zero, the number" " of available cpus will be used",
        nohash=True,
    )
    
class tckSIFT2OutputSpec(TraitedSpec):
    out_file=File(argstr="%s", desc="output text file containing the weighting factor for each streamline")
    
class tckSIFT2(CommandLine):
    """
    Interface with mrtrix3 package
    Spherical-deconvolution informed filtering of tractograms - sift2
    Optimise per-streamline cross-section multipliers to match a whole-brain tractogram to fixel-wise fibre densities
    """
    _cmd="tcksift2"
    input_spec=tckSIFT2InputSpec
    output_spec=tckSIFT2OutputSpec
    
    def _list_outputs(self):
        outputs=self.output_spec().get()
        outputs["out_file"] = os.path.abspath(self.inputs.out_file)
        return outputs

### Generate tissue type specific masks

Use `fsl` algorithm to generate tissue specific `.mif` file with `5ttgen`

In [ ]:
#example
#usage
gen5tt = Generate5tt()
gen5tt.inputs.in_file = ''  # T1 acpc aligned nifti
gen5tt.inputs.algorithm = 'fsl'  # fsl or freesurfer, fsl is preferred in my opinion
gen5tt.inputs.out_file = '5tt.mif'
# gen5tt.inputs.args = '-premasked'  # indicating a brain mask has already been applied to input image
gen5tt.cmdline

### Tweak Atlas labels with look up tables.

mrtrix3's `labelconvert` and `labelsgmfix` if using freesurfer atlas.

Use the nipype interface for `LabelConvert`, and declare a mrtrix3 path to its `/share/mrtrix3/labelconvert/` folder where the target connectome lookup tabel for output images reside. Available files include: 

    - aal.txt
    - aal2.txt
    - fs_default.txt
    - hcpmmp1_ordered.txt
    - hcpmmp1_original.txt
    - lpba40.txt
    - fs2lobes_cinginc_convert.txt
    - fs2lobes_cinginc_labels.txt
    - fs2lobes_cingsep_convert.txt
    - fs2lobes_cingsep_labels.txt
    
Please refer to https://mrtrix.readthedocs.io/en/latest/quantitative_structural_connectivity/labelconvert_tutorial.html#labelconvert-tutorial for detailed explanation.

In [ ]:
mrt_labels = LabelConvert()
mrt_labels.inputs.in_file = '/Users/xxie/lab/atlases/BN_Atlas_246_2mm.nii'
mrt_labels.inputs.in_lut = '/Users/xxie/lab/atlases/BN_Atlas_246_LUT.txt'
mrt_labels.inputs.cmdline

`tck2connectome` for connectivity matrix and distance adjacency matrix

In [ ]:
#exporti
class MakeConnectomeInputSpec(CommandLineInputSpec):
    """
    Specifying inputs to mrtrix3's tck2connectome
    """
    in_file = File(
        exists=True, mandatory=True, argstr="%s", position=-3, desc="input tck file"
    )
    in_parc = File(
        exists=True, argstr="%s", position=-2, desc="parcellation file"
    )
    out_file = File(
        argstr="%s",
        mandatory=True,
        position=-1,
        desc="output file connectivity csv file",
    )
    nthreads = traits.Int(
        argstr="-nthreads %d",
        desc="number of threads. if zero, the number" " of available cpus will be used",
        nohash=True,
    )
    in_weights = File(
        exists=True,
        argstr="-tck_weights_in %s",
        desc="specify a text scalar file containing the streamline weights",
    )
    scale_length=traits.Bool(
        argstr="-scale_length",
        desc="scale each contribution to the connectome edge by the length of the streamline"
    )
    stat_edge=traits.Enum(
        "sum",
        "mean",
        "min",
        "max",
        argstr="-stat_edge %s",
        desc="statistic for combining the values from all streamlines in an edge into a single scale value for that edge (options are: sum,mean,min,max;default=sum)"
    )
    
class MakeConnectomeOutputSpec(TraitedSpec):
    out_file = File(argstr="%s", desc="output connectome csv file")
    
class MakeConnectome(CommandLine):
    """
    mrtrix3's tck2connectome interface
    """
    _cmd="tck2connectome"
    input_spec = MakeConnectomeInputSpec
    output_spec = MakeConnectomeOutputSpec
    
    def _list_outputs(self):
        outputs=self.output_spec().get()
        outputs["out_file"] = os.path.abspath(self.inputs.out_file)
        return outputs